# Players dataframes for the same period t

En este script nos dedicaremos a crear una base de datos limpia segmentada por hitters y pitchers. Se exportarán dichas bases de datos contemplando respectivamente a los jugadores que son agentes libres, a los que no son y a todos los jugadores. Las secciones dentro del script son:

- **Visualización del contenido de las bases de datos.**
- **Limpieza de la base de datos y exportación.**
- **Creación de indicador de si el jugador es agente libre.**

Importemos los modulos necesarios así como especificar la configuración deseada.

In [104]:
import pandas as pd
import numpy as np
import math
import os
import warnings
print('Modulos importados')

Modulos importados


In [105]:
# Configuraciones
warnings.filterwarnings('ignore')
# Reduzcamos el número de línea a leer
pd.options.display.max_rows = 5

In [106]:
# Directorio de trabajo
print("Directorio de trabajo previo: " + str(os.getcwd()))
# Cambiemoslo
os.chdir('/home/usuario/Documentos/Github/Proyectos/MLB_HN/')

Directorio de trabajo previo: /home/usuario/Documentos/Github/Proyectos/MLB_HN


In [107]:
# Veamos el directorio actual de trabajo
print(os.getcwd())
# El directorio anterior es el correcto, pero si no lo fuese, hacemos lo sigueinte:
path = '/home/usuario/Documentos/Github/Proyectos/MLB_HN'
print("Nuevo directorio de trabajo: " + str(os.chdir(path)))

/home/usuario/Documentos/Github/Proyectos/MLB_HN
Nuevo directorio de trabajo: None


## Visualización de las bases de datos

Basta con ver el contenido de las base de datos de un año para observar qué variables contienen. Escojamos el año 2012.

A continuación, se mostrará el contenido de las distintas bases de datos sobre los *bateadores*, *pitchers*, *salarios de los agentes libres* y *salarios de los todos los jugadores*. Esto para determinar el proceso de limpieza que se llevará a cabo.

In [108]:
# Rutas de los archivos del año 2012
free_agents_2012 = 'Data/Free_Agents/free_agents_2012.csv'
hitting_2012 = 'Data/Not_All_Variables/Statistics/Hitting/hitting_2012.csv'
pitching_2012 = 'Data/Not_All_Variables/Statistics/Pitching/pitching_2012.csv'
salary_2012 = 'Data/Not_All_Variables/Salary/salary_2012.csv'
teams_etl_2012 = 'ETL_Data/Agent/Teams/free_agents_team_2012.csv'

# Importando los dataframes
df_free_agent_auxiliar_2012 = pd.read_csv(free_agents_2012)
df_hitting_auxiliar_2012 = pd.read_csv(hitting_2012)
df_pitching_auxiliar_2012 = pd.read_csv(pitching_2012)
df_salary_auxiliar_2012 = pd.read_csv(salary_2012)
df_teams_etl_2012 = pd.read_csv(teams_etl_2012)

### Agentes libres

Veamos primero el dataframe

In [109]:
df_free_agent_auxiliar_2012.head()

,Rank,Player,Year,Pos,Status,Team From,Team From To,YRS,Value,AAV
0,1,Albert Pujols,2012,DH,UFA,STL,LAA,10,"$240,000,000","$24,000,000"
1,2,Prince Fielder,2012,DH,UFA,MIL,DET,9,"$214,000,000","$23,777,778"
2,3,Jose Reyes,2012,SS,UFA,NYM,MIA,6,"$106,000,000","$17,666,667"
3,4,C.J. Wilson,2012,SP,UFA,TEX,LAA,5,"$77,500,000","$15,500,000"
4,5,Mark Buehrle,2012,SP,UFA,CHW,MIA,4,"$58,000,000","$14,500,000"


### Hitters

Veamos el dataframe

In [110]:
df_hitting_auxiliar_2012.head()

,Rank,Player,Pos,Team,GP,GP%,GS,GS%,AB,H,...,HR,RBI,AVG,OBP,SLG,OPS,WAR,TVS,Cash2023,Unnamed: 21
0,NaN,Derek Jeter,SS,NYY,159,0.982,158,0.975,683,216,...,15,58,0.316,0.362,0.429,0.791,2.16,23.61,$0,NaN
1,NaN,Miguel Cabrera,1B,DET,161,0.994,161,0.994,622,205,...,44,139,0.330,0.393,0.606,0.999,7.14,96.42,"$32,000,000",NaN
2,NaN,Robinson Cano,2B,NYY,161,0.994,159,0.982,627,196,...,33,94,0.313,0.379,0.550,0.929,8.44,98.76,$0,NaN
3,NaN,Everth Cabrera,SS,SD,230,0.710,218,0.673,796,196,...,4,48,0.246,0.324,0.324,0.648,3.56,82.66,$0,NaN
4,NaN,Adrian Beltre,3B,TEX,156,0.963,152,0.938,604,194,...,36,102,0.321,0.359,0.561,0.921,7.24,88.47,$0,NaN


In [111]:
df_hitting_auxiliar_2012.columns

Index(['Rank', 'Player', 'Pos', 'Team', 'GP', 'GP%', 'GS', 'GS%', 'AB', 'H',
       '2B', '3B', 'HR', 'RBI', 'AVG', 'OBP', 'SLG', 'OPS', 'WAR', 'TVS',
       'Cash2023', 'Unnamed: 21'],
      dtype='object')

Los términos en la base de datos no se traducirán para evitar malentendidos en la traducción.

- **Pos**: Player position.
- **Team**: Team acronym.
- **GP**: Games played.
- **GP%**: Games played %.
- **AB**: At bats.
- **H**: Hitting.
- **HR**: Home runs.
- **RBI**: Runs batted in.
- **AVG**: Batting average.
- **OPS**: Onebase plus slugging%.

Se omitirá la columna *Cash2022* puesto que no es de interés para el trabajo el valor del jugador en la actualidad puesto que hay agentes libres que ya se han retirado en años posteriores.

## Pitchers

In [112]:
df_pitching_auxiliar_2012.head()

,Rank,Player,Pos,Team,GP,GS,IP,H,R,ER,...,SO,W,L,SV,WHIP,ERA,WAR,TVS,Cash2023,Unnamed: 20
0,NaN,R.A. Dickey,SP,NYM,34,33,233.7,192,78,71,...,230,20,6,0,1.05,2.74,5.69,97.27,$0,NaN
1,NaN,Felix Hernandez,SP,SEA,33,33,232.0,209,84,79,...,223,13,9,0,1.14,3.07,5.26,85.20,$0,NaN
2,NaN,James Shields,SP,TB,33,33,227.7,209,103,89,...,223,15,10,0,1.17,3.52,2.54,79.41,$0,NaN
3,NaN,Clayton Kershaw,SP,LAD,34,33,227.7,170,70,64,...,229,14,9,0,1.02,2.53,6.43,95.71,"$20,000,000",NaN
4,NaN,Hiroki Kuroda,SP,NYY,33,33,219.7,205,86,81,...,167,16,11,0,1.16,3.32,5.27,81.85,$0,NaN


In [113]:
df_pitching_auxiliar_2012.columns

Index(['Rank', 'Player', 'Pos', 'Team', 'GP', 'GS', 'IP', 'H', 'R', 'ER', 'BB',
       'SO', 'W', 'L', 'SV', 'WHIP', 'ERA', 'WAR', 'TVS', 'Cash2023',
       'Unnamed: 20'],
      dtype='object')

#### Notación.

Veamos a qué se refieren algunos términos

- **Pos**: Player position.
- **Team**: Team acronym.
- **GP**: Games played.
- **GS**: Games started.
- **IP**: Inning pitched.
- **H**: Hits.
- **R**: Runs.
- **ER**: Earned runs.
- **BB**: Walks.
- **SO**: Strikeouts.
- **W**: Wins.
- **L**: Losses-
- **SV**: Saves.
- **WHIP**: WHIP.
- **ERA**: Earned runs average.

Por razones análogas, se descartará la columna *Cash2022*.

### Salarios
En este caso, hay muchas menos variables que en las anteriores bases de datos

In [114]:
df_salary_auxiliar_2012.head()

,Rank,Player,Year,Pos,Team,BaseSalary,SigningBonus,Payroll Salary,Adj Salary,Salary%,...,AAV,CONT YR,CONT VALUE,Earnings,FA Year,Sign Age,Age,Weight,Height,Unnamed: 20
0,NaN,Alex Rodriguez,2012,DH,NYY,"$29,000,000","$1,000,000","$30,000,000","$30,000,000",0.132,...,"$27,500,000",10,"$275,000,000","$321,290,700",2018,32,37,230,"0'0""",NaN
1,NaN,C.C. Sabathia,2012,SP,NYY,"$23,000,000","$1,285,714","$24,285,714","$24,285,714",0.107,...,"$23,000,000",7,"$161,000,000","$127,285,714",2016,28,31,300,"6'6""",NaN
2,NaN,Vernon Wells,2012,LF,LAA,"$21,000,000","$3,187,500","$24,187,500","$24,187,500",0.154,...,"$18,000,000",7,"$126,000,000","$102,521,000",2015,29,33,0,"0'0""",NaN
3,NaN,Johan Santana,2012,SP,NYM,"$24,000,000",$0,"$24,000,000","$24,000,000",0.228,...,"$22,916,667",6,"$137,500,000","$148,560,000",2014,28,33,155,"6'0""",NaN
4,NaN,Mark Teixeira,2012,1B,NYY,"$22,500,000","$625,000","$23,125,000","$23,125,000",0.101,...,"$22,500,000",8,"$180,000,000","$127,650,000",2017,28,32,225,"6'3""",NaN


In [115]:
df_salary_auxiliar_2012.columns

Index(['Rank', 'Player', 'Year', 'Pos', 'Team', 'BaseSalary', 'SigningBonus',
       'Payroll Salary', 'Adj Salary', 'Salary%', 'Cash', 'AAV', 'CONT YR',
       'CONT VALUE', 'Earnings', 'FA Year', 'Sign Age', 'Age', 'Weight',
       'Height', 'Unnamed: 20'],
      dtype='object')

- **BaseSalary**: A base salary is the minimum amount you can expect to earn in exchange for your time or services. This is the amount earned before benefits, bonuses, or compensation is added.
- **Payroll Salary**: Payroll is the compensation a business must pay to its employees for a set period and on a given date plus signing bonus
- **Adj Salary**: Adjusted Salary means the regular salary, wages and commissions, if any, payable to a Participant by the Company for the Participant's service, excluding any bonuses or other compensation.

### Teams ETL

Esta base de datos sobre los equipos es bajo el proceso ETL

In [116]:
df_teams_etl_2012.head()

,Equipo,Cantidad_agentes_libres,Valor_contrato_total,Acronimo,Victorias,Juegos totales,Playoffs,Pennants won,WS ganadas,Promedio_victorias
0,Los Angeles Angels,4,321150000,LAA,89,162,10,1,1,0.549383
1,Detroit Tigers,3,221000000,DET,88,162,14,11,4,0.543210
2,Miami Marlins,6,203300000,MIA,69,162,2,2,2,0.425926
3,Philadelphia Phillies,7,57650000,PHI,81,162,14,7,2,0.500000
4,Los Angeles Dodgers,9,44651311,LAD,86,162,26,25,6,0.530864


### Equipos por estado

In [117]:
states = 'Data/Teams/team_states.csv'
df_states = pd.read_csv(states)

In [118]:
df_states.head()

,Estado,Cantidad de equipos
0,Alabama,0
1,Alaska,0
2,Arizona,1
3,Arkansas,0
4,California,5


### Acrónimos

Nos servirá como llave intermedia para unificar las bases de datos de los equipos

In [119]:
acronym = 'Data/Teams/team_acronym.csv'
df_acronym = pd.read_csv(acronym)

In [120]:
df_acronym.head()

,Equipo,Acronimo,Estado
0,Arizona Diamondbacks,ARI,Arizona
1,Atlanta Braves,ATL,Georgia
2,Baltimore Orioles,BAL,Maryland
3,Boston Red Sox,BOS,Massachusetts
4,Chicago Cubs,CHC,Illinois


Unamos esta dataframe con el de los equipos por estado

In [121]:
acronym_state = pd.merge(df_states, df_acronym, on = 'Estado')

In [122]:
acronym_state.head()

,Estado,Cantidad de equipos,Equipo,Acronimo
0,Arizona,1,Arizona Diamondbacks,ARI
1,California,5,Los Angeles Angels,LAA
2,California,5,Los Angeles Dodgers,LAD
3,California,5,Oakland Athletics,OAK
4,California,5,San Diego Padres,SD


En este caso, el nombre de las variables es claro

## Algoritmo para la creación de las bases de datos

A continuaicón, se optimizará el código para que se puedan obtener los *dataframes* anteriores para un cojuntos de datos de años secuenciales, como es nuestro caso

In [123]:
# Auxiliares:
free_agents = 'Data/Free_Agents/free_agents_'
hitting = 'Data/Not_All_Variables/Statistics/Hitting/hitting_'
pitching = 'Data/Not_All_Variables/Statistics/Pitching/pitching_'
salary = 'Data/Not_All_Variables/Salary/salary_'
teams = 'ETL_Data/Agent/Teams/free_agents_team_'
csv = '.csv'
period = 12
# Originales:
df_free_agents = [None]*period
df_hitting = [None]*period
df_pitching = [None]*period
df_salary = [None]*period
df_teams = [None]*period
# Copias:
df_free_agents_copy = [None]*period
df_hitting_copy = [None]*period
df_pitching_copy = [None]*period
df_salary_copy = [None]*period
df_teams_copy = [None]*period
# Producto final:
df_pitchers = [None]*period
df_hitters = [None]*period
df_pitchers_free_agents = [None]*period
df_hitters_free_agents = [None]*period
df_pitchers_no_free_agents = [None]*period
df_hitters_no_free_agents = [None]*period
df_panel_hitters = [None]*period
df_panel_pitchers = [None]*period

Leamos todos los archivos y creemos las copias

In [124]:
for year in range(0,period):    
    df_free_agents[year] = pd.read_csv(free_agents + str(2011 + year) + csv)
    df_hitting[year] = pd.read_csv(hitting + str(2011 + year) + csv)
    df_pitching[year] = pd.read_csv(pitching + str(2011 + year) + csv)
    df_salary[year] = pd.read_csv(salary + str(2011 + year) + csv)
    df_teams[year] = pd.read_csv(teams + str(2011 + year) + csv)
    
    df_free_agents_copy[year] = df_free_agents[year].copy()
    df_hitting_copy[year] = df_hitting[year].copy()
    df_pitching_copy[year] = df_pitching[year].copy()
    df_salary_copy[year] = df_salary[year].copy()
    df_teams_copy[year] = pd.read_csv(teams + str(2011 + year) + csv)

Tratemos las bases de datos por separado. Sin embargo, a todas les quitaremos la columna de rango y *Cash2023*.

Como no queremos que se repita la columna del año de la temporada de la base de datos, borremos la columna de *Year* de la base  de datos de los agentes libres. Como los años del contrato aparecen en la base de datos sobre los salarios, se prefiere conservar dicha columna en la base de datos de salarios puesto que esta base de datos es más general que la de los agentes libres, razón por la que se borrará de esta última base de datos. 

El equipo al que se cambia el agente libre está señalado por la columna del equipo en la base de datos de salarios y la estadísticas deportivas por lo que se borrará *Team From To* de la base de datos de los agentes libres. 

Como nos importan los salarios para este analisis, quitaremos la columna de los equipos en las bases de datos sobre las estadísticas deportivas de todos los jugadores, así como la posición que ocupan.

In [125]:
for year in range(0,period):
    # Drop columns:
    if any(name in df_free_agents_copy[year].columns for name in ['Rank','Pos','Year','YRS','Team From To']):
        df_free_agents_copy[year].drop('Rank', axis = 1, inplace = True)
        df_free_agents_copy[year].drop('Year', axis = 1, inplace = True)
        df_free_agents_copy[year].drop('Pos', axis = 1, inplace = True)
        df_free_agents_copy[year].drop('YRS', axis = 1, inplace = True)
        df_free_agents_copy[year].drop('Team From To', axis = 1, inplace = True)
    if 'Rank' in df_salary_copy[year].columns:
        df_salary_copy[year].drop('Rank', axis = 1, inplace = True)
    if any(name in df_hitting_copy[year].columns for name in ['Rank','Year','Cash2023','Team','Pos']):
        df_hitting_copy[year].drop('Rank', axis = 1, inplace = True)
        df_hitting_copy[year].drop('Cash2023', axis = 1, inplace = True)
        df_hitting_copy[year].drop('Team', axis = 1, inplace = True)
        df_hitting_copy[year].drop('Pos', axis = 1, inplace = True)
    if any(name in df_pitching_copy[year].columns for name in ['Rank','Year','Cash2023','Team','Pos']):
        df_pitching_copy[year].drop('Rank', axis = 1, inplace = True)
        df_pitching_copy[year].drop('Cash2023', axis = 1, inplace = True)
        df_pitching_copy[year].drop('Team', axis = 1, inplace = True)
        df_pitching_copy[year].drop('Pos', axis = 1, inplace = True)

Debido a que aparecen columnas que inician con el  nombre *Unnamed*, tendremos que borrarlas con algún método general, el cual se muestra a continuación:

In [126]:
for year in range(0,period):
    # Base de datos de agentes libres:
    df_free_agents_copy[year].drop(df_free_agents_copy[year].columns[df_free_agents_copy[year].columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
    
    # Base de datos de los salarios:
    df_salary_copy[year].drop(df_salary_copy[year].columns[df_salary_copy[year].columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
    
    # Base de datos de los hitters:
    df_hitting_copy[year].drop(df_hitting_copy[year].columns[df_hitting_copy[year].columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
    
    # Base de datos de los pitchers:
    df_pitching_copy[year].drop(df_pitching_copy[year].columns[df_pitching_copy[year].columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)

Verifiquemos que ya no se encuentran dichas columnas molestas

In [127]:
df_free_agents_copy[9].columns

Index(['Player', 'Status', 'Team From', 'Value', 'AAV'], dtype='object')

In [128]:
df_salary_copy[11].columns

Index(['Player', 'Year', 'Pos', 'Team', 'BaseSalary', 'SigningBonus',
       'Payroll Salary', 'Adj Salary', 'Salary%', 'Cash', 'AAV', 'CONT YR',
       'CONT VALUE', 'Earnings', 'FA Year', 'Sign Age', 'Age', 'Weight',
       'Height'],
      dtype='object')

In [129]:
df_hitting_copy[2].columns

Index(['Player', 'GP', 'GP%', 'GS', 'GS%', 'AB', 'H', '2B', '3B', 'HR', 'RBI',
       'AVG', 'OBP', 'SLG', 'OPS', 'WAR', 'TVS'],
      dtype='object')

In [130]:
df_pitching_copy[5].columns

Index(['Player', 'GP', 'GS', 'IP', 'H', 'R', 'ER', 'BB', 'SO', 'W', 'L', 'SV',
       'WHIP', 'ERA', 'WAR', 'TVS'],
      dtype='object')

#### Agentes libres

No se conservará el equipo al que es contratado el agente libre puesto que esta información también la contiene la base de datos que facilita más el tratamiento _ETL_.

In [131]:
for year in range(0,period):
    df_free_agents_copy[year] = df_free_agents_copy[year].rename(columns = {'Player':'Jugador',
                            'Status':'Status_agente_libre', 'Team From':'Equipo_anterior',
                            'Value':'Valor_contrato', 'AAV':'Valor_promedio_contrato'})
    
    free_agents_aux_1 = df_free_agents_copy[year]['Valor_contrato'].str.replace("$","")
    free_agents_aux_2 = free_agents_aux_1.str.replace(",","")
    free_agents_aux_3 = df_free_agents_copy[year]['Valor_promedio_contrato'].str.replace("$","")
    free_agents_aux_4 = free_agents_aux_3.str.replace(",","")
    df_free_agents_copy[year]['Valor_contrato'] = free_agents_aux_2
    df_free_agents_copy[year]['Valor_promedio_contrato'] = free_agents_aux_4
    
    df_free_agents_copy[year]['Valor_contrato'] = pd.to_numeric(df_free_agents_copy[year]['Valor_contrato'])
    df_free_agents_copy[year]['Valor_promedio_contrato'] = pd.to_numeric(df_free_agents_copy[year]['Valor_promedio_contrato'])

Observemos las dimensiones de las bases de datos como referencia

In [132]:
for year in range(0,period):
    print(df_free_agents_copy[year].shape)

(1, 5)
(108, 5)
(213, 5)
(208, 5)
(221, 5)
(241, 5)
(100, 5)
(98, 5)
(105, 5)
(118, 5)
(141, 5)
(137, 5)


También el tipo de datos que contiene cada columna

In [133]:
df_free_agents_copy[1].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Jugador                  108 non-null    object
 1   Status_agente_libre      108 non-null    object
 2   Equipo_anterior          108 non-null    object
 3   Valor_contrato           108 non-null    int64 
 4   Valor_promedio_contrato  108 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 4.3+ KB


#### Salarios

Como los salarios irán con las bases de datos de los _hitters_ y _pitchers_ es que se hará su proceso _ETL_ antes.

In [134]:
for year in range(0,period):
    # Cambio de nombres
    df_salary_copy[year] = df_salary_copy[year].rename(columns = {'Player':'Jugador',
                            'BaseSalary':'Sueldo_base', 'SigningBonus':'Bono_por_firma',
                            'Payroll Salary':'Sueldo_regular', 'Adj Salary':'Sueldo_ajustado',
                            'CONT YR':'Anios_de_contrato', 'CONT VALUE':'Valor_del_contrato',
                            'Earnings':'Ganancias', 'FA Year':'Anio_de_agente_libre',
                            'Sign Age':'Edad_al_firmar', 'Age':'Edad', 'Weight':'Peso',
                            'Height':'Altura', 'Year':'Anio', 'Pos':'Posicion',
                            'Salary%':'Sueldo_porcentual', 'Cash':'Pago_efectivo',
                            'AAV':'Valor_contrato_promedio', 'Team':'Acronimo'})
    
    # Tranformando al tipo de dato apropiado
    salary_aux_1 = df_salary_copy[year]['Sueldo_base'].str.replace("$","")
    salary_aux_2 = salary_aux_1.str.replace(",","")
    df_salary_copy[year]['Sueldo_base'] = salary_aux_2
    df_salary_copy[year]['Sueldo_base'] = pd.to_numeric(df_salary_copy[year]['Sueldo_base'])
    
    salary_aux_3 = df_salary_copy[year]['Sueldo_regular'].str.replace("$","")
    salary_aux_4 = salary_aux_3.str.replace(",","")
    df_salary_copy[year]['Sueldo_regular'] = salary_aux_4
    df_salary_copy[year]['Sueldo_regular'] = pd.to_numeric(df_salary_copy[year]['Sueldo_regular'])
    
    salary_aux_5 = df_salary_copy[year]['Sueldo_ajustado'].str.replace("$","")
    salary_aux_6 = salary_aux_5.str.replace(",","")
    df_salary_copy[year]['Sueldo_ajustado'] = salary_aux_6
    df_salary_copy[year]['Sueldo_ajustado'] = pd.to_numeric(df_salary_copy[year]['Sueldo_ajustado'])
    
    salary_aux_7 = df_salary_copy[year]['Valor_del_contrato'].str.replace("$","")
    salary_aux_8 = salary_aux_7.str.replace(",","")
    df_salary_copy[year]['Valor_del_contrato'] = salary_aux_8
    df_salary_copy[year]['Valor_del_contrato'] = pd.to_numeric(df_salary_copy[year]['Valor_del_contrato'])
    
    salary_aux_9 = df_salary_copy[year]['Bono_por_firma'].str.replace("$","")
    salary_aux_10 = salary_aux_9.str.replace(",","")
    df_salary_copy[year]['Bono_por_firma'] = salary_aux_10
    df_salary_copy[year]['Bono_por_firma'] = pd.to_numeric(df_salary_copy[year]['Bono_por_firma'])
    
    salary_aux_11 = df_salary_copy[year]['Ganancias'].str.replace("$","")
    salary_aux_12 = salary_aux_11.str.replace(",","")
    df_salary_copy[year]['Ganancias'] = salary_aux_12
    df_salary_copy[year]['Ganancias'] = pd.to_numeric(df_salary_copy[year]['Ganancias'])
    
    salary_aux_13 = df_salary_copy[year]['Pago_efectivo'].str.replace("$","")
    salary_aux_14 = salary_aux_13.str.replace(",","")
    df_salary_copy[year]['Pago_efectivo'] = salary_aux_14
    df_salary_copy[year]['Pago_efectivo'] = pd.to_numeric(df_salary_copy[year]['Pago_efectivo'])
    
    salary_aux_15 = df_salary_copy[year]['Valor_contrato_promedio'].str.replace("$","")
    salary_aux_16 = salary_aux_15.str.replace(",","")
    df_salary_copy[year]['Valor_contrato_promedio'] = salary_aux_16
    df_salary_copy[year]['Valor_contrato_promedio'] = pd.to_numeric(df_salary_copy[year]['Valor_contrato_promedio'])
    
    salary_aux_17 = df_salary_copy[year]['Altura'].str.replace("\"","")
    salary_aux_18 = salary_aux_17.str.replace("'","")
    df_salary_copy[year]['Altura'] = salary_aux_18
    df_salary_copy[year]['Altura'] = pd.to_numeric(df_salary_copy[year]['Altura'])/10
    # SUstitullamos los xeros
    height_mean = df_salary_copy[year]['Altura'].mean(skipna=True)
    df_salary_copy[year]['Altura'] = df_salary_copy[year].Altura.mask(df_salary_copy[year].Altura == 0, height_mean)
    
    df_salary_copy[year]['Anio_de_agente_libre'] = pd.to_numeric(df_salary_copy[year]['Anio_de_agente_libre'])
    df_salary_copy[year]['Anios_de_contrato'] = pd.to_numeric(df_salary_copy[year]['Anios_de_contrato'])
    df_salary_copy[year]['Edad'] = pd.to_numeric(df_salary_copy[year]['Edad'])

Por algunas particularidades de la base de datos, las columna que contiene la edad al firmar se tratará por separado aprovechando que la mayoría de los datos incorrectos tienen una longitud mayor a dos.

In [135]:
for year in range (0,period):
    df_salary_copy[year]['Edad_al_firmar'] = df_salary_copy[year]['Edad_al_firmar'].map(str)

    for edad in range(0,df_salary_copy[year].shape[0]):
        # String es mayor que 0:
        if len(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]) == 2:
            df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(df_salary_copy[year]['Edad_al_firmar'].iloc[edad])
            
        # String es menor o igual que 0:
        elif len(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]) != 2:
            # Si la columna de la edad contiene datos correctos
            if df_salary_copy[year]['Edad'].iloc[edad] > 0:
                if df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad] == 0:
                    ag_year = year + 2011 + 1
                else:
                    ag_year = df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad]
                # Get first year of contract
                ini_year = ag_year - df_salary_copy[year]['Anios_de_contrato'].iloc[edad]
                # Años desde el el año inicial
                dif_years = year + 2011 - ini_year
                # Edad al firmar:
                sign_age = df_salary_copy[year]['Edad'].iloc[edad] - dif_years
                # Cambio de dato:
                df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(sign_age)
                
            # Si la columna de edad no contiene un dato coherente
            else:
                # Cambio de dato:
                df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(18)    
    
        # Entero  menor a 0:
        if df_salary_copy[year]['Edad_al_firmar'].iloc[edad] < 0:
            # Si la columna de la edad contiene datos correctos
            if df_salary_copy[year]['Edad'].iloc[edad] > 0:
                if df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad] == 0:
                    ag_year = year + 2011 + 1
                else:
                    ag_year = df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad]
                # Get first year of contract
                ini_year = ag_year - df_salary_copy[year]['Anios_de_contrato'].iloc[edad]
                # Años desde el el año inicial
                dif_years = year + 2011 - ini_year
                # Edad al firmar:
                sign_age = df_salary_copy[year]['Edad'].iloc[edad] - dif_years
                # Cambio de dato:
                df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(sign_age)
                
            # Si la columna de edad no contiene un dato coherente
            else:
                # Cambio de dato:
                df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(18)
         
    # Transformemos los datos a enteros
    df_salary_copy[year]['Edad_al_firmar'] = pd.to_numeric(df_salary_copy[year]['Edad_al_firmar'])

Podemos verificar si se limpiaron adecuadamente las celdas de la columna de edades al firmar. Esto, al filtrar los datos que sean distintos a enteros y al observar si se pudo transformar toda la columna al tipo entero.

In [136]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year]['Edad_al_firmar'].shape[0]):
        if type(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]) != np.int64:
            print(type(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]))

In [137]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year]['Edad_al_firmar'].shape[0]):
        if df_salary_copy[year]['Edad_al_firmar'].iloc[edad] < 0:
            print(df_salary_copy[year]['Edad_al_firmar'].iloc[edad])

In [138]:
#for year in range(0,period):
#    print(type(df_salary_copy[year][['Edad_al_firmar']].info()))

In [139]:
#for year in range(0,period):
#    print(year)
#    for edad in range(0,df_salary_copy[year]['Edad_al_firmar'].shape[0]):
#        print(str(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]) + ' ' + str(edad))

Por otro lado, falta corregir las entradas de las columnas de las edades que tengan valores menores a cero. Esto se hará de acuerdo al resto de columnas

In [140]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year]['Edad'].shape[0]):
        if df_salary_copy[year]['Edad'].iloc[edad] < 0:
            print(year)
            print(df_salary_copy[year]['Edad'].iloc[edad])

0
-9
1
-8
2
-7


In [141]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year].shape[0]):
        # Condición para imputar:
        if df_salary_copy[year]['Edad'].iloc[edad] <= 0:
            # Si no se indica si tendrá año de agencia libre:
            if df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad] == 0:
                        ag_year = year + 2011 + 1
            # Si tendrá año de agencia libre
            else:
                ag_year = df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad]
            # Get first year of contract
            ini_year = ag_year - df_salary_copy[year]['Anios_de_contrato'].iloc[edad]
            # Años desde el el año inicial
            dif_years = year + 2011 - ini_year
            # Edad en la temporada:
            seasson_age = df_salary_copy[year]['Edad_al_firmar'].iloc[edad] + dif_years
            # Asignación
            df_salary_copy[year]['Edad'].iloc[edad] = seasson_age

Comprobemos que no hay ninguna edad negativa

In [142]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year]['Edad'].shape[0]):
        if df_salary_copy[year]['Edad'].iloc[edad] < 0:
            print(year)
            print(str(df_salary_copy[year]['Edad'].iloc[edad]) + ' ' + str(edad))

Con la imputación de datos ya se puede crear la columna que contiene la antiguedad del agente libre bajo el contrato

In [143]:
for year in range(0,period):
    df_salary_copy[year]['Antiguedad'] = df_salary_copy[year]['Edad'] - df_salary_copy[year]['Edad_al_firmar']

Por último, convirtamos la columna del año a string para que se entienda como una categoría y no una variable numérica

In [144]:
for year in range(0,period):
    df_salary_copy[year]['Anio'] = df_salary_copy[year]['Anio'].map(str)

In [145]:
df_salary_copy[5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1045 entries, 0 to 1044
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Jugador                  1045 non-null   object 
 1   Anio                     1045 non-null   object 
 2   Posicion                 1045 non-null   object 
 3   Acronimo                 1045 non-null   object 
 4   Sueldo_base              1045 non-null   int64  
 5   Bono_por_firma           1045 non-null   int64  
 6   Sueldo_regular           1045 non-null   int64  
 7   Sueldo_ajustado          1045 non-null   int64  
 8   Sueldo_porcentual        1045 non-null   float64
 9   Pago_efectivo            1045 non-null   int64  
 10  Valor_contrato_promedio  1045 non-null   int64  
 11  Anios_de_contrato        1045 non-null   int64  
 12  Valor_del_contrato       1045 non-null   int64  
 13  Ganancias                1045 non-null   int64  
 14  Anio_de_agente_libre    

#### Hitters

In [146]:
for year in range(0,period):
    # Cambio de nombres
    df_hitting_copy[year] = df_hitting_copy[year].rename(columns = {'Player':'Jugador',
                            'GP':'Juegos', 'GP%':'Porcentaje_juegos',
                            'AB':'At-bats', 'H':'Bateos', 'GS':'Juegos_iniciados',
                            'GS%':'Porcentaje_juegos_iniciados', 'RBI':'Runs-batted-in',
                            'HR':'Home-runs', 'AVG':'Bateos_promedio',
                            '2B':'Dobles', '3B':'Triples', 'OPS':'Porcentaje_On-base-plus-slugging',
                            'SLG':'Porcentaje_slugging', 'OBP':'Porcentaje_on-base'})

In [147]:
for year in range(0,period):
    print(df_hitting_copy[year].shape)

(521, 17)
(667, 17)
(734, 17)
(778, 17)
(809, 17)
(834, 17)
(810, 17)
(825, 17)
(850, 17)
(451, 17)
(753, 17)
(479, 17)


In [148]:
df_hitting_copy[5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 834 entries, 0 to 833
Data columns (total 17 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Jugador                           834 non-null    object 
 1   Juegos                            834 non-null    int64  
 2   Porcentaje_juegos                 834 non-null    float64
 3   Juegos_iniciados                  834 non-null    int64  
 4   Porcentaje_juegos_iniciados       834 non-null    float64
 5   At-bats                           834 non-null    int64  
 6   Bateos                            834 non-null    int64  
 7   Dobles                            834 non-null    int64  
 8   Triples                           834 non-null    int64  
 9   Home-runs                         834 non-null    int64  
 10  Runs-batted-in                    834 non-null    int64  
 11  Bateos_promedio                   834 non-null    float64
 12  Porcenta

In [149]:
df_hitting_copy[5].columns

Index(['Jugador', 'Juegos', 'Porcentaje_juegos', 'Juegos_iniciados',
       'Porcentaje_juegos_iniciados', 'At-bats', 'Bateos', 'Dobles', 'Triples',
       'Home-runs', 'Runs-batted-in', 'Bateos_promedio', 'Porcentaje_on-base',
       'Porcentaje_slugging', 'Porcentaje_On-base-plus-slugging', 'WAR',
       'TVS'],
      dtype='object')

#### Pitchers

In [150]:
for year in range(0,period):
    # Cambio de nombres
    df_pitching_copy[year] = df_pitching_copy[year].rename(columns = {'Player':'Jugador',
                             'GP':'Juegos', 'GS':'Juegos_iniciados', 'IP':'Inning_pitched',
                             'H':'Bateos_en_contra', 'R':'Carreras_en_contra', 'ER':'Carreras_ganadas',
                             'BB':'Walks', 'SO':'Strike-outs', 'W':'Wins', 'L':'Losses',
                             'SV':'Saves'})

In [151]:
for year in range(0,period):
    print(df_pitching_copy[year].shape)

(324, 16)
(425, 16)
(485, 16)
(537, 16)
(586, 16)
(592, 16)
(581, 16)
(644, 16)
(668, 16)
(542, 16)
(611, 16)
(585, 16)


In [152]:
df_pitching_copy[5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 592 entries, 0 to 591
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Jugador             592 non-null    object 
 1   Juegos              592 non-null    int64  
 2   Juegos_iniciados    592 non-null    int64  
 3   Inning_pitched      592 non-null    float64
 4   Bateos_en_contra    592 non-null    int64  
 5   Carreras_en_contra  592 non-null    int64  
 6   Carreras_ganadas    592 non-null    int64  
 7   Walks               592 non-null    int64  
 8   Strike-outs         592 non-null    int64  
 9   Wins                592 non-null    int64  
 10  Losses              592 non-null    int64  
 11  Saves               592 non-null    int64  
 12  WHIP                592 non-null    float64
 13  ERA                 592 non-null    float64
 14  WAR                 587 non-null    float64
 15  TVS                 592 non-null    float64
dtypes: float

## Agregación de variables sugeridas por artículos

Las primeras variables que agregaremos son el cuadrado de todas las estadísticas deportivas, así como las siguientes variables:

- DOMINANCE = $Strike-outs/(9*Inning \; Pitched)$
- CONTROL = $Walks/(9*Inning \; Pitched)$
- COMMAND = $Strike-outs/Walks$

In [153]:
for year in range(0,period):
    df_pitching_copy[year]['Dominio'] = df_pitching_copy[year]['Strike-outs']/(9*df_pitching_copy[year]['Inning_pitched'])
    df_pitching_copy[year]['Control'] = df_pitching_copy[year]['Walks']/(9*df_pitching_copy[year]['Inning_pitched'])
    df_pitching_copy[year]['Comando'] = df_pitching_copy[year]['Strike-outs']/df_pitching_copy[year]['Walks']

In [154]:
for year in range(0,period):
    print(df_pitching_copy[year].shape)

(324, 19)
(425, 19)
(485, 19)
(537, 19)
(586, 19)
(592, 19)
(581, 19)
(644, 19)
(668, 19)
(542, 19)
(611, 19)
(585, 19)


In [155]:
df_pitching_copy[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485 entries, 0 to 484
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Jugador             485 non-null    object 
 1   Juegos              485 non-null    int64  
 2   Juegos_iniciados    485 non-null    int64  
 3   Inning_pitched      485 non-null    float64
 4   Bateos_en_contra    485 non-null    int64  
 5   Carreras_en_contra  485 non-null    int64  
 6   Carreras_ganadas    485 non-null    int64  
 7   Walks               485 non-null    int64  
 8   Strike-outs         485 non-null    int64  
 9   Wins                485 non-null    int64  
 10  Losses              485 non-null    int64  
 11  Saves               485 non-null    int64  
 12  WHIP                485 non-null    float64
 13  ERA                 485 non-null    float64
 14  WAR                 435 non-null    float64
 15  TVS                 485 non-null    float64
 16  Dominio 

Podemos verificar qué entradas poseen valores infinitos en la base de datos

In [156]:
"""
for year in range(0,period):
    print(str(2011 + year))
    for name in df_pitching_copy[year].columns:
        print(name)
        if type(name) != str:
            for element in range(0,len(df_pitching_copy[year][name])):
                if math.isinf(df_pitching_copy[year][name].iloc[element]) == True:
                    print(str(element) +  '  ' + str(df_pitching_copy[year][name].iloc[element]))
    print("")
"""

'\nfor year in range(0,period):\n    print(str(2011 + year))\n    for name in df_pitching_copy[year].columns:\n        print(name)\n        if type(name) != str:\n            for element in range(0,len(df_pitching_copy[year][name])):\n                if math.isinf(df_pitching_copy[year][name].iloc[element]) == True:\n                    print(str(element) +  \'  \' + str(df_pitching_copy[year][name].iloc[element]))\n    print("")\n'

Siguiendo la sugerencia de algunos artículos, obtengamos el logaritmo de los salarios

In [157]:
for year in range(0,period):
    df_salary_copy[year]['ln_Sueldo_base'] = np.log(df_salary_copy[year]['Sueldo_base'])
    df_salary_copy[year]['ln_Sueldo_ajustado'] = np.log(df_salary_copy[year]['Sueldo_ajustado'])
    df_salary_copy[year]['ln_Sueldo_regular'] = np.log(df_salary_copy[year]['Sueldo_regular'])

In [158]:
df_salary_copy[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1011 entries, 0 to 1010
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Jugador                  1011 non-null   object 
 1   Anio                     1011 non-null   object 
 2   Posicion                 1011 non-null   object 
 3   Acronimo                 1011 non-null   object 
 4   Sueldo_base              1011 non-null   int64  
 5   Bono_por_firma           1011 non-null   int64  
 6   Sueldo_regular           1011 non-null   int64  
 7   Sueldo_ajustado          1011 non-null   int64  
 8   Sueldo_porcentual        1011 non-null   float64
 9   Pago_efectivo            1011 non-null   int64  
 10  Valor_contrato_promedio  1011 non-null   int64  
 11  Anios_de_contrato        1011 non-null   int64  
 12  Valor_del_contrato       1011 non-null   int64  
 13  Ganancias                1011 non-null   int64  
 14  Anio_de_agente_libre    

Debido a que hay columnas con datos tipo _Nan_ o _NULL_, optaremos por imputarlos.

Mientras que los valores infinitos generados por las nuevas variables se sustituirán dependediendo del caso:

- 0/0: 0
- num/0: Máximo de la columna correspondientefijarán

In [159]:
for year in range(0,period):
    # Salaries
    mean_hgt = df_salary_copy[year].loc[df_salary_copy[year]['Altura'] > 4.9].Altura.mean()
    mean_wgh = df_salary_copy[year].loc[df_salary_copy[year]['Peso'] > 0].Peso.mean()
    df_salary_copy[year]['Altura'].fillna(value = mean_hgt, inplace = True)
    df_salary_copy[year]['Altura'].mask(df_salary_copy[year]['Altura'] <= 4.9, mean_hgt, inplace = True)
    df_salary_copy[year]['Peso'].fillna(value = mean_wgh, inplace = True)
    df_salary_copy[year]['Peso'].mask(df_salary_copy[year]['Peso'] <= 0, mean_wgh, inplace = True)
    
    # Pitchers
    mean_war = df_pitching_copy[year].loc[df_pitching_copy[year]['WAR'] > 0].WAR.mean()
    mean_dom = df_pitching_copy[year].loc[df_pitching_copy[year]['Dominio'] > 0].Dominio.mean()
    mean_con = df_pitching_copy[year].loc[df_pitching_copy[year]['Control'] > 0].Control.mean()
    mean_com = df_pitching_copy[year].loc[df_pitching_copy[year]['Comando'] > 0].Comando.mean()
    df_pitching_copy[year]['WAR'].fillna(value = mean_war, inplace = True)
    df_pitching_copy[year]['WAR'].mask(df_pitching_copy[year]['WAR'] <= 0, mean_war, inplace = True)
    df_pitching_copy[year]['Dominio'].fillna(value = mean_dom, inplace = True)
    df_pitching_copy[year]['Dominio'].mask(df_pitching_copy[year]['Dominio'] <= 0, mean_dom, inplace = True)
    df_pitching_copy[year]['Control'].fillna(value = mean_con, inplace = True)
    df_pitching_copy[year]['Control'].mask(df_pitching_copy[year]['Control'] <= 0, mean_con, inplace = True)
    df_pitching_copy[year]['Comando'].fillna(value = mean_com, inplace = True)
    df_pitching_copy[year]['Comando'].mask(df_pitching_copy[year]['Comando'] <= 0, mean_com, inplace = True)
    
    # Hitters
    mean_war = df_hitting_copy[year].loc[df_hitting_copy[year]['WAR'] > 0].WAR.mean()
    df_hitting_copy[year]['WAR'].fillna(value = mean_war, inplace = True)
    df_hitting_copy[year]['WAR'].mask(df_hitting_copy[year]['WAR'] <= 0, mean_war, inplace = True)

In [160]:
for year in range(0,period):   
    # Condiciones
    con_dom_1 = df_pitching_copy[year]['Strike-outs'] == 0
    con_con_1 = df_pitching_copy[year]['Walks'] == 0
    con_com_1 = df_pitching_copy[year]['Strike-outs'] == 0
                 
    # Imputación caso 0/0
    df_pitching_copy[year].loc[con_dom_1, "Dominio"] = 0
    df_pitching_copy[year].loc[con_con_1, "Control"] = 0
    df_pitching_copy[year].loc[con_com_1, "Comando"] = 0

In [161]:
for year in range(0,period):   
    # Máximos
    max_dom = df_pitching_copy[year]['Strike-outs'].max()/9
    max_con = df_pitching_copy[year]['Walks'].max()/9
    max_com = df_pitching_copy[year]['Strike-outs'].max()
    
    # Cambianfdo infinitos a NaNs
    df_pitching_copy[year]["Dominio"].replace([np.inf, -np.inf], np.nan, inplace = True)
    df_pitching_copy[year]["Control"].replace([np.inf, -np.inf], np.nan, inplace = True)
    df_pitching_copy[year]["Comando"].replace([np.inf, -np.inf], np.nan, inplace = True)
    
    # Imputación
    df_pitching_copy[year]['Dominio'].fillna(value = max_dom, inplace = True)
    df_pitching_copy[year]['Control'].fillna(value = max_con, inplace = True)
    df_pitching_copy[year]['Comando'].fillna(value = max_com, inplace = True)

Verifiquemos que ya no haya problemas con valores infinitos

In [162]:
"""
for year in range(0,period):
    print(str(2011 + year))
    for name in df_pitching_copy[year].columns:
        print(name)
        if type(name) != str:
            for element in range(0,len(df_pitching_copy[year][name])):
                if math.isinf(df_pitching_copy[year][name].iloc[element]) == True:
                    print(str(element) +  '  ' + str(df_pitching_copy[year][name].iloc[element]))
    print("")
"""

'\nfor year in range(0,period):\n    print(str(2011 + year))\n    for name in df_pitching_copy[year].columns:\n        print(name)\n        if type(name) != str:\n            for element in range(0,len(df_pitching_copy[year][name])):\n                if math.isinf(df_pitching_copy[year][name].iloc[element]) == True:\n                    print(str(element) +  \'  \' + str(df_pitching_copy[year][name].iloc[element]))\n    print("")\n'

Así mismo, contemos los valores *NaN* que queden presentes

In [163]:
for year in range(0,period):
    print('Año: ' + str(2011 + year))
    print('Hitters:')
    df_hitting_copy[year].isna().sum()
    print('Pitchers:')
    df_pitching_copy[year].isna().sum()
    print('Free agents:')
    df_free_agents_copy[year].isna().sum()
    print('Salaries:')
    df_salary_copy[year].isna().sum()
    print("")

Año: 2011
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2012
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2013
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2014
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2015
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2016
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2017
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2018
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2019
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2020
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2021
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2022
Hitters:
Pitchers:
Free agents:
Salaries:



Ahora, repitamos este proceso para la base de datos de los salarios.

In [164]:
salary_names = ['ln_Sueldo_ajustado', 'ln_Sueldo_base', 'ln_Sueldo_regular']

In [165]:
for name in salary_names:
    print(name)
    
    for year in range(0,period):
        print(str(2011 + year))
        for element in range(0,len(df_salary_copy[year][name])):
            if df_salary_copy[year][name].iloc[element] <= 0:
                print(str(element) +  '  ' + str(df_salary_copy[year][name].iloc[element]))
        print("")

ln_Sueldo_ajustado
2011

2012

2013

2014

2015

2016

2017

2018
72  -inf
188  -inf

2019

2020

2021
6  -inf
56  -inf
166  -inf
274  -inf

2022
201  -inf

ln_Sueldo_base
2011

2012

2013

2014

2015

2016

2017

2018

2019

2020

2021

2022

ln_Sueldo_regular
2011

2012

2013

2014

2015

2016

2017

2018

2019

2020

2021

2022



Al inspecsionar los errores nos damos cuenta que solo se desconocen los salarios ajustados y los fijaron a $0$. Usaremos el logaritmo base 10 del salario regular para sustituir dicho valor.

In [166]:
for year in range(0,period):
    df_salary_copy[year]['ln_Sueldo_ajustado'].mask(df_salary_copy[year]['ln_Sueldo_ajustado'] < 0,
                                                    df_salary_copy[year]['ln_Sueldo_regular'],
                                                    inplace = True)

In [167]:
for year in range(0,period):
    print(str(2011 + year))
    for element in range(0,len(df_salary_copy[year]['ln_Sueldo_ajustado'])):
        if df_salary_copy[year]['ln_Sueldo_ajustado'].iloc[element] <= 0:
            print(str(element) +  '  ' + str(df_salary_copy[year]['ln_Sueldo_ajustado'].iloc[element]))
    print("")

2011

2012

2013

2014

2015

2016

2017

2018

2019

2020

2021

2022



In [168]:
for year in range(0,period):
    print("Ajustado: " + str(df_salary_copy[year]['ln_Sueldo_ajustado'].mean())
          + 'n'
          + 'Regular: ' + str(df_salary_copy[year]['ln_Sueldo_regular'].mean()))

Ajustado: 13.786883876822797nRegular: 13.888874386883515
Ajustado: 13.836339875004654nRegular: 13.909807816176343
Ajustado: 13.547035081556775nRegular: 13.9779431913153
Ajustado: 13.66411021920866nRegular: 14.080738990191257
Ajustado: 13.49415866539551nRegular: 13.99344595993313
Ajustado: 13.478992231977141nRegular: 14.0261405412466
Ajustado: 13.575627559754176nRegular: 14.080502234724484
Ajustado: 13.562343599267946nRegular: 14.08078787299267
Ajustado: 13.52136464369358nRegular: 14.066559580453122
Ajustado: 13.029933516088823nRegular: 14.217470030278857
Ajustado: 13.688221153044282nRegular: 14.18813695050661
Ajustado: 13.923634888060981nRegular: 14.351533324360805


En efecto, ya no hay valores _NaN_ o _infinitos_.

Con el objetivo de hacer más eficiente la creación de las variables al cuadrado, lo haremos extrayendo el índice de las columnas de interés

In [169]:
df_hitting_copy[0].columns

Index(['Jugador', 'Juegos', 'Porcentaje_juegos', 'Juegos_iniciados',
       'Porcentaje_juegos_iniciados', 'At-bats', 'Bateos', 'Dobles', 'Triples',
       'Home-runs', 'Runs-batted-in', 'Bateos_promedio', 'Porcentaje_on-base',
       'Porcentaje_slugging', 'Porcentaje_On-base-plus-slugging', 'WAR',
       'TVS'],
      dtype='object')

In [170]:
df_pitching_copy[1].columns

Index(['Jugador', 'Juegos', 'Juegos_iniciados', 'Inning_pitched',
       'Bateos_en_contra', 'Carreras_en_contra', 'Carreras_ganadas', 'Walks',
       'Strike-outs', 'Wins', 'Losses', 'Saves', 'WHIP', 'ERA', 'WAR', 'TVS',
       'Dominio', 'Control', 'Comando'],
      dtype='object')

In [171]:
def get_col_indices(df, names):
    return df.columns.get_indexer(names)

In [172]:
hitting_names = ['Juegos_iniciados', 'Porcentaje_juegos_iniciados', 'At-bats', 'Bateos',
                  'Dobles', 'Triples', 'Home-runs', 'Runs-batted-in', 'Bateos_promedio',
                  'Porcentaje_on-base', 'Porcentaje_slugging', 'TVS',
                  'Porcentaje_On-base-plus-slugging', 'WAR']	
pitching_names = ['Inning_pitched', 'Bateos_en_contra', 'Carreras_en_contra',
                  'Carreras_ganadas', 'Walks', 'Strike-outs', 'Wins', 'Losses',
                  'Saves', 'WHIP', 'ERA', 'WAR', 'TVS', 'Dominio', 'Control',
                  'Comando']

Con el objetivo de simplificar el código, verifiquemos si todos los índices en cada base de datos son los mismos

In [173]:
print('Hitters:')
for year in range(0,period):
    print(get_col_indices(df_hitting_copy[year], hitting_names))
    
print('Pitchers:')
for year in range(0,period):
    print(get_col_indices(df_pitching_copy[year], pitching_names))

Hitters:
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
Pitchers:
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3

In [174]:
hitting_indexes = list(get_col_indices(df_hitting_copy[0], hitting_names))
pitching_indexes = list(get_col_indices(df_pitching_copy[0], pitching_names))

In [175]:
for year in range(0,period):
    # Hitters:
    for hitter_name in hitting_indexes:
        df_hitting_copy[year][df_hitting_copy[year].columns[hitter_name] + '_2'] = np.power(df_hitting_copy[year][df_hitting_copy[year].columns[hitter_name]], 2)
    # Pitchers:
    for pitcher_name in pitching_indexes:
        df_pitching_copy[year][df_pitching_copy[year].columns[pitcher_name] + '_2'] = np.power(df_pitching_copy[year][df_pitching_copy[year].columns[pitcher_name]], 2)

Apreciemos el resultado final

In [176]:
df_hitting_copy[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 734 entries, 0 to 733
Data columns (total 31 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Jugador                             734 non-null    object 
 1   Juegos                              734 non-null    int64  
 2   Porcentaje_juegos                   734 non-null    float64
 3   Juegos_iniciados                    734 non-null    int64  
 4   Porcentaje_juegos_iniciados         734 non-null    float64
 5   At-bats                             734 non-null    int64  
 6   Bateos                              734 non-null    int64  
 7   Dobles                              734 non-null    int64  
 8   Triples                             734 non-null    int64  
 9   Home-runs                           734 non-null    int64  
 10  Runs-batted-in                      734 non-null    int64  
 11  Bateos_promedio                     734 non-n

In [177]:
df_pitching_copy[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485 entries, 0 to 484
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Jugador               485 non-null    object 
 1   Juegos                485 non-null    int64  
 2   Juegos_iniciados      485 non-null    int64  
 3   Inning_pitched        485 non-null    float64
 4   Bateos_en_contra      485 non-null    int64  
 5   Carreras_en_contra    485 non-null    int64  
 6   Carreras_ganadas      485 non-null    int64  
 7   Walks                 485 non-null    int64  
 8   Strike-outs           485 non-null    int64  
 9   Wins                  485 non-null    int64  
 10  Losses                485 non-null    int64  
 11  Saves                 485 non-null    int64  
 12  WHIP                  485 non-null    float64
 13  ERA                   485 non-null    float64
 14  WAR                   485 non-null    float64
 15  TVS                   4

## Unión de las bases de datos
### Datos agregados por equipo

Solo resta añadir los datos relevantes al equipo al que pertenece cada jugador considerando la base de datos de la cantidad de equipos por estado

In [178]:
df_teams_copy[7].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Equipo                   28 non-null     object 
 1   Cantidad_agentes_libres  28 non-null     int64  
 2   Valor_contrato_total     28 non-null     int64  
 3   Acronimo                 28 non-null     object 
 4   Victorias                28 non-null     int64  
 5   Juegos totales           28 non-null     int64  
 6   Playoffs                 28 non-null     int64  
 7   Pennants won             28 non-null     int64  
 8   WS ganadas               28 non-null     int64  
 9   Promedio_victorias       28 non-null     float64
dtypes: float64(1), int64(7), object(2)
memory usage: 2.3+ KB


In [179]:
acronym_state.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 29
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Estado               30 non-null     object
 1   Cantidad de equipos  30 non-null     int64 
 2   Equipo               30 non-null     object
 3   Acronimo             30 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.2+ KB


In [180]:
for year in range(0,period):
    df_teams_copy[year] = pd.merge(df_teams_copy[year], acronym_state, on = ['Equipo','Acronimo'])

In [181]:
df_teams_copy[7].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28 entries, 0 to 27
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Equipo                   28 non-null     object 
 1   Cantidad_agentes_libres  28 non-null     int64  
 2   Valor_contrato_total     28 non-null     int64  
 3   Acronimo                 28 non-null     object 
 4   Victorias                28 non-null     int64  
 5   Juegos totales           28 non-null     int64  
 6   Playoffs                 28 non-null     int64  
 7   Pennants won             28 non-null     int64  
 8   WS ganadas               28 non-null     int64  
 9   Promedio_victorias       28 non-null     float64
 10  Estado                   28 non-null     object 
 11  Cantidad de equipos      28 non-null     int64  
dtypes: float64(1), int64(8), object(3)
memory usage: 2.8+ KB


In [182]:
df_salary_copy[7].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1036 entries, 0 to 1035
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Jugador                  1036 non-null   object 
 1   Anio                     1036 non-null   object 
 2   Posicion                 1036 non-null   object 
 3   Acronimo                 1036 non-null   object 
 4   Sueldo_base              1036 non-null   int64  
 5   Bono_por_firma           1036 non-null   int64  
 6   Sueldo_regular           1036 non-null   int64  
 7   Sueldo_ajustado          1036 non-null   int64  
 8   Sueldo_porcentual        1036 non-null   float64
 9   Pago_efectivo            1036 non-null   int64  
 10  Valor_contrato_promedio  1036 non-null   int64  
 11  Anios_de_contrato        1036 non-null   int64  
 12  Valor_del_contrato       1036 non-null   int64  
 13  Ganancias                1036 non-null   int64  
 14  Anio_de_agente_libre    

Ahora, unamos las bases de datos sobre los equipos a las bases de datos de los salarios

In [183]:
for year in range(0,period):
    df_salary_copy[year] = pd.merge(df_teams_copy[year], df_salary_copy[year], on = 'Acronimo')

In [184]:
df_salary_copy[0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 0 to 39
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Equipo                   40 non-null     object 
 1   Cantidad_agentes_libres  40 non-null     int64  
 2   Valor_contrato_total     40 non-null     int64  
 3   Acronimo                 40 non-null     object 
 4   Victorias                40 non-null     int64  
 5   Juegos totales           40 non-null     int64  
 6   Playoffs                 40 non-null     int64  
 7   Pennants won             40 non-null     int64  
 8   WS ganadas               40 non-null     int64  
 9   Promedio_victorias       40 non-null     float64
 10  Estado                   40 non-null     object 
 11  Cantidad de equipos      40 non-null     int64  
 12  Jugador                  40 non-null     object 
 13  Anio                     40 non-null     object 
 14  Posicion                 40 

Debido a que la mayoría de los jugadores juega tanto en la ofensiva como la defensiva es que tenemos que borrar los duplicados de la columna de la posición.

In [185]:
for year in range(0,period):
    df_hitting_copy[year] = pd.merge(df_hitting_copy[year], df_salary_copy[year], on = 'Jugador')
    df_pitching_copy[year] = pd.merge(df_pitching_copy[year], df_salary_copy[year], on = 'Jugador')

In [186]:
for year in range(0,period):
    df_pitching_copy[year]['Porcentaje_juegos'] = df_pitching_copy[year]['Juegos']/df_pitching_copy[year]['Juegos totales']

In [187]:
df_hitting_copy[3].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 668 entries, 0 to 667
Data columns (total 64 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Jugador                             668 non-null    object 
 1   Juegos                              668 non-null    int64  
 2   Porcentaje_juegos                   668 non-null    float64
 3   Juegos_iniciados                    668 non-null    int64  
 4   Porcentaje_juegos_iniciados         668 non-null    float64
 5   At-bats                             668 non-null    int64  
 6   Bateos                              668 non-null    int64  
 7   Dobles                              668 non-null    int64  
 8   Triples                             668 non-null    int64  
 9   Home-runs                           668 non-null    int64  
 10  Runs-batted-in                      668 non-null    int64  
 11  Bateos_promedio                     668 non-n

In [188]:
df_pitching_copy[3].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 498 entries, 0 to 497
Data columns (total 69 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Jugador                  498 non-null    object 
 1   Juegos                   498 non-null    int64  
 2   Juegos_iniciados         498 non-null    int64  
 3   Inning_pitched           498 non-null    float64
 4   Bateos_en_contra         498 non-null    int64  
 5   Carreras_en_contra       498 non-null    int64  
 6   Carreras_ganadas         498 non-null    int64  
 7   Walks                    498 non-null    int64  
 8   Strike-outs              498 non-null    int64  
 9   Wins                     498 non-null    int64  
 10  Losses                   498 non-null    int64  
 11  Saves                    498 non-null    int64  
 12  WHIP                     498 non-null    float64
 13  ERA                      498 non-null    float64
 14  WAR                      4

Para facilitar la observación de las trnasformaciones de manera más eficiente, ordenemos alfabéticamente la base de datos de acuerdo al nombre de las columnas.

In [189]:
for year in range(0,period):
    # Ordenando alfabéticamente
    df_salary_copy[year].sort_index(axis = 1, inplace = True)
    df_hitting_copy[year].sort_index(axis = 1, inplace = True)
    df_pitching_copy[year].sort_index(axis = 1, inplace = True)
    df_free_agents_copy[year].sort_index(axis = 1, inplace = True)
    
    # Reiniciando los índices
    df_salary_copy[year].reset_index(drop = True, inplace = True)
    df_hitting_copy[year].reset_index(drop = True, inplace = True)
    df_pitching_copy[year].reset_index(drop = True, inplace = True)
    df_free_agents_copy[year].reset_index(drop = True, inplace = True)

## Segmentación por Agentes libres

Separaremos los pitchers y hitters en dos grupos:

- Agentes libres.
- No agentes libres.

In [190]:
for year in range(0,period):
    # Filtrando los agentes libres
    df_hitters_free_agents[year] = pd.merge(df_free_agents_copy[year],
                                            df_hitting_copy[year], on = 'Jugador')
    df_pitchers_free_agents[year] = pd.merge(df_free_agents_copy[year],
                                             df_pitching_copy[year], on = 'Jugador')
    # FIltrando los que no son agentes libres
    df_hitters_no_free_agents[year] = df_hitting_copy[year][~df_hitting_copy[year].Jugador.isin(df_hitters_free_agents[year].Jugador)]
    df_pitchers_no_free_agents[year] = df_pitching_copy[year][~df_pitching_copy[year].Jugador.isin(df_pitchers_free_agents[year].Jugador)]
    
    # Reiniciando el índice
    df_hitters_free_agents[year] = df_hitters_free_agents[year].reindex(sorted(df_hitters_free_agents[year].columns), axis=1)
    df_pitchers_free_agents[year] = df_pitchers_free_agents[year].reindex(sorted(df_pitchers_free_agents[year].columns), axis=1)
    df_hitters_no_free_agents[year] = df_hitters_no_free_agents[year].reindex(sorted(df_hitters_no_free_agents[year].columns), axis=1)
    df_pitchers_no_free_agents[year] = df_pitchers_no_free_agents[year].reindex(sorted(df_pitchers_no_free_agents[year].columns), axis=1)    

Veamos los contenidos de las nuevas bases de datos

In [191]:
print("FA - Hitters:")
df_hitters_free_agents[9].info()
print("\n FA - Pitchers:")
df_pitchers_free_agents[9].info()
print("\n No FA - Hitters:")
df_hitters_no_free_agents[9].info()
print("\n No FA - Hitters:")
df_pitchers_no_free_agents[9].info()

FA - Hitters:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 39 entries, 0 to 38
Data columns (total 68 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Acronimo                            39 non-null     object 
 1   Altura                              39 non-null     float64
 2   Anio                                39 non-null     object 
 3   Anio_de_agente_libre                39 non-null     int64  
 4   Anios_de_contrato                   39 non-null     int64  
 5   Antiguedad                          39 non-null     int64  
 6   At-bats                             39 non-null     int64  
 7   At-bats_2                           39 non-null     int64  
 8   Bateos                              39 non-null     int64  
 9   Bateos_2                            39 non-null     int64  
 10  Bateos_promedio                     39 non-null     float64
 11  Bateos_promedio_2                

Por último, para facilitar futuras aplicaciones, pasemos todos los nombres de las columnas a miníscula

In [192]:
for year in range(0,period):
    df_hitters_free_agents[year].rename(columns=str.lower)
    df_pitchers_free_agents[year].rename(columns=str.lower)
    df_hitters_no_free_agents[year].rename(columns=str.lower)
    df_pitchers_no_free_agents[year].rename(columns=str.lower)

In [193]:
for year in range(0,period):    
    # Exportemos los dataframes por separado
    df_hitters_free_agents[year].to_csv('ETL_Data/Agent/First_Year_Contract/Period_t/Free_Agent/Hitters/free_agents_batters_' + str(2011 + year) + '.csv', index = False)
    df_pitchers_free_agents[year].to_csv('ETL_Data/Agent/First_Year_Contract/Period_t/Free_Agent/Pitchers/free_agents_pitchers_' + str(2011 + year) + '.csv', index = False)
    df_hitters_no_free_agents[year].to_csv('ETL_Data/Agent/First_Year_Contract/Period_t/No_Free_Agent/Hitters/no_free_agents_batters_' + str(2011 + year) + '.csv', index = False)
    df_pitchers_no_free_agents[year].to_csv('ETL_Data/Agent/First_Year_Contract/Period_t/No_Free_Agent/Pitchers/no_free_agents_pitchers_' + str(2011 + year) + '.csv', index = False)

### Etiquetas para los agentes libres

Crearemos un etiqueta para indicar si el pitcher o hitter es  un agente libre o no.

In [194]:
for year in range(0,period):
    # Condiciones
    condicion_hitter = [df_hitting_copy[year].Jugador.isin(df_free_agents_copy[year].Jugador)]
    condicion_pitcher = [df_pitching_copy[year].Jugador.isin(df_free_agents_copy[year].Jugador)]
    
    # Etiquetas
    etiquetas = ['Si']
    
    df_hitting_copy[year]['Agente_libre'] = np.select(condicion_hitter, etiquetas, default = 'No')
    df_pitching_copy[year]['Agente_libre'] = np.select(condicion_pitcher, etiquetas, default = 'No')
    
    df_hitting_copy[year] = df_hitting_copy[year].reindex(sorted(df_hitting_copy[year].columns), axis=1)
    df_pitching_copy[year] = df_pitching_copy[year].reindex(sorted(df_pitching_copy[year].columns), axis=1)

Verifiquemos los resultados

In [195]:
print("Hitters:")
df_hitting_copy[3].info()
print("\n Pitchers:")
df_pitching_copy[3].info()

Hitters:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668 entries, 0 to 667
Data columns (total 65 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Acronimo                            668 non-null    object 
 1   Agente_libre                        668 non-null    object 
 2   Altura                              668 non-null    float64
 3   Anio                                668 non-null    object 
 4   Anio_de_agente_libre                668 non-null    int64  
 5   Anios_de_contrato                   668 non-null    int64  
 6   Antiguedad                          668 non-null    int64  
 7   At-bats                             668 non-null    int64  
 8   At-bats_2                           668 non-null    int64  
 9   Bateos                              668 non-null    int64  
 10  Bateos_2                            668 non-null    int64  
 11  Bateos_promedio                     

In [196]:
for year in range(0,period):
    df_hitting_copy[year].rename(columns = str.lower)
    df_pitching_copy[year].rename(columns = str.lower)

In [197]:
for year in range(0,period):
    # Exportemos los dataframes
    df_hitting_copy[year].to_csv('ETL_Data/Agent/First_Year_Contract/Period_t/Hitters/All_Hitters/hitters_' + str(2011 + year) + '.csv', index = False)
    df_pitching_copy[year].to_csv('ETL_Data/Agent/First_Year_Contract/Period_t/Pitchers/All_Pitchers/pitchers_' + str(2011 + year) + '.csv', index = False)

## Panel Data

Con el objetivo de contar con una base de datos en estructura panel, uniremos las bases de datos

In [198]:
# Inicialización del panel
df_panel_all_hitter = df_hitting_copy[0]
df_panel_all_pitcher = df_pitching_copy[0]

for year in range(1,period):
    # Hitter
    df_panel_all_hitter = pd.concat([df_panel_all_hitter, df_hitting_copy[year]])
    
    # Pitcher
    df_panel_all_pitcher = pd.concat([df_panel_all_pitcher, df_pitching_copy[year]])

Veamos las estadísticas descriptivas de los panel

In [199]:
df_panel_all_hitter.describe()

,Altura,Anio_de_agente_libre,Anios_de_contrato,Antiguedad,At-bats,At-bats_2,Bateos,Bateos_2,Bateos_promedio,Bateos_promedio_2,...,Valor_contrato_promedio,Valor_contrato_total,Valor_del_contrato,Victorias,WAR,WAR_2,WS ganadas,ln_Sueldo_ajustado,ln_Sueldo_base,ln_Sueldo_regular
count,6814.00000,6814.000000,6814.000000,6814.000000,6814.000000,6814.000000,6814.000000,6814.000000,6814.000000,6814.000000,...,6.814000e+03,6.814000e+03,6.814000e+03,6814.000000,6814.000000,6814.000000,6814.000000,6814.000000,6814.000000,6814.000000
mean,6.18491,1131.184913,1.870267,0.915175,189.854124,76155.117112,48.053566,5330.265776,0.193387,0.053172,...,3.988427e+06,6.049538e+07,1.627472e+07,78.457147,1.787627,5.209351,3.846933,13.768998,14.200866,14.219590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75%,6.30000,2022.000000,1.000000,1.000000,346.750000,120235.750000,86.000000,7396.000000,0.264000,0.069696,...,5.000000e+06,7.696250e+07,8.000000e+06,90.000000,2.024341,4.097956,5.000000,15.176487,15.308415,15.334747
max,6.90000,2035.000000,14.000000,9.000000,684.000000,467856.000000,225.000000,50625.000000,1.000000,1.000000,...,3.554167e+07,5.807000e+08,4.265000e+08,111.000000,10.680000,114.062400,27.000000,17.440000,17.399029,17.487583


In [200]:
df_panel_all_pitcher.describe()

,Altura,Anio_de_agente_libre,Anios_de_contrato,Antiguedad,Bateos_en_contra,Bateos_en_contra_2,Bono_por_firma,Cantidad de equipos,Cantidad_agentes_libres,Carreras_en_contra,...,WHIP,WHIP_2,WS ganadas,Walks,Walks_2,Wins,Wins_2,ln_Sueldo_ajustado,ln_Sueldo_base,ln_Sueldo_regular
count,5764.000000,5764.000000,5764.000000,5764.000000,5764.000000,5764.00000,5.764000e+03,5764.000000,5764.000000,5764.000000,...,5764.000000,5764.000000,5764.000000,5764.000000,5764.000000,5764.000000,5764.000000,5764.000000,5764.000000,5764.000000
mean,6.257708,1104.708362,1.462873,0.719639,61.563498,6808.24601,7.653541e+04,2.126301,5.386711,31.416031,...,1.401273,2.459711,3.852706,22.209403,818.097675,3.737162,30.765267,13.488598,14.003133,14.016606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75%,6.400000,2022.000000,1.000000,1.000000,81.000000,6561.00000,0.000000e+00,2.000000,7.000000,43.000000,...,1.510000,2.280100,5.000000,31.000000,961.000000,5.000000,25.000000,14.626441,14.827111,14.862830
max,6.900000,2029.000000,9.000000,8.000000,314.000000,98596.00000,1.000000e+07,5.000000,18.000000,162.000000,...,21.000000,441.000000,27.000000,106.000000,11236.000000,22.000000,484.000000,17.584433,17.584433,17.584433


In [201]:
df_panel_all_hitter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6814 entries, 0 to 396
Data columns (total 65 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Acronimo                            6814 non-null   object 
 1   Agente_libre                        6814 non-null   object 
 2   Altura                              6814 non-null   float64
 3   Anio                                6814 non-null   object 
 4   Anio_de_agente_libre                6814 non-null   int64  
 5   Anios_de_contrato                   6814 non-null   int64  
 6   Antiguedad                          6814 non-null   int64  
 7   At-bats                             6814 non-null   int64  
 8   At-bats_2                           6814 non-null   int64  
 9   Bateos                              6814 non-null   int64  
 10  Bateos_2                            6814 non-null   int64  
 11  Bateos_promedio                     6814 non

In [202]:
df_panel_all_pitcher.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5764 entries, 0 to 545
Data columns (total 70 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Acronimo                 5764 non-null   object 
 1   Agente_libre             5764 non-null   object 
 2   Altura                   5764 non-null   float64
 3   Anio                     5764 non-null   object 
 4   Anio_de_agente_libre     5764 non-null   int64  
 5   Anios_de_contrato        5764 non-null   int64  
 6   Antiguedad               5764 non-null   int64  
 7   Bateos_en_contra         5764 non-null   int64  
 8   Bateos_en_contra_2       5764 non-null   int64  
 9   Bono_por_firma           5764 non-null   int64  
 10  Cantidad de equipos      5764 non-null   int64  
 11  Cantidad_agentes_libres  5764 non-null   int64  
 12  Carreras_en_contra       5764 non-null   int64  
 13  Carreras_en_contra_2     5764 non-null   int64  
 14  Carreras_ganadas         

Verifquemos que no haya problemas con valores *NaN* o *infinitos*

Valores *NaN*:

In [203]:
for name in df_panel_all_hitter.columns:
    print(name)
    if type(name) != str:
        for element in range(0,len(df_panel_all_hitter[name])):
            if pd.isna(df_panel_all_hitter[name].iloc[element]) == True:
                print(str(element) +  '  ' + str(df_panel_all_hitter[name].iloc[element]))

Acronimo
Agente_libre
Altura
Anio
Anio_de_agente_libre
Anios_de_contrato
Antiguedad
At-bats
At-bats_2
Bateos
Bateos_2
Bateos_promedio
Bateos_promedio_2
Bono_por_firma
Cantidad de equipos
Cantidad_agentes_libres
Dobles
Dobles_2
Edad
Edad_al_firmar
Equipo
Estado
Ganancias
Home-runs
Home-runs_2
Juegos
Juegos totales
Juegos_iniciados
Juegos_iniciados_2
Jugador
Pago_efectivo
Pennants won
Peso
Playoffs
Porcentaje_On-base-plus-slugging
Porcentaje_On-base-plus-slugging_2
Porcentaje_juegos
Porcentaje_juegos_iniciados
Porcentaje_juegos_iniciados_2
Porcentaje_on-base
Porcentaje_on-base_2
Porcentaje_slugging
Porcentaje_slugging_2
Posicion
Promedio_victorias
Runs-batted-in
Runs-batted-in_2
Sueldo_ajustado
Sueldo_base
Sueldo_porcentual
Sueldo_regular
TVS
TVS_2
Triples
Triples_2
Valor_contrato_promedio
Valor_contrato_total
Valor_del_contrato
Victorias
WAR
WAR_2
WS ganadas
ln_Sueldo_ajustado
ln_Sueldo_base
ln_Sueldo_regular


In [204]:
for name in df_panel_all_pitcher.columns:
    print(name)
    if type(name) != str:
        for element in range(0,len(df_panel_all_pitcher[name])):
            if pd.isna(df_panel_all_pitcher[name].iloc[element]) == True:
                print(str(element) +  '  ' + str(df_panel_all_pitcher[name].iloc[element]))

Acronimo
Agente_libre
Altura
Anio
Anio_de_agente_libre
Anios_de_contrato
Antiguedad
Bateos_en_contra
Bateos_en_contra_2
Bono_por_firma
Cantidad de equipos
Cantidad_agentes_libres
Carreras_en_contra
Carreras_en_contra_2
Carreras_ganadas
Carreras_ganadas_2
Comando
Comando_2
Control
Control_2
Dominio
Dominio_2
ERA
ERA_2
Edad
Edad_al_firmar
Equipo
Estado
Ganancias
Inning_pitched
Inning_pitched_2
Juegos
Juegos totales
Juegos_iniciados
Jugador
Losses
Losses_2
Pago_efectivo
Pennants won
Peso
Playoffs
Porcentaje_juegos
Posicion
Promedio_victorias
Saves
Saves_2
Strike-outs
Strike-outs_2
Sueldo_ajustado
Sueldo_base
Sueldo_porcentual
Sueldo_regular
TVS
TVS_2
Valor_contrato_promedio
Valor_contrato_total
Valor_del_contrato
Victorias
WAR
WAR_2
WHIP
WHIP_2
WS ganadas
Walks
Walks_2
Wins
Wins_2
ln_Sueldo_ajustado
ln_Sueldo_base
ln_Sueldo_regular


Valores *infinitos*

In [205]:
for name in df_panel_all_hitter.columns:
    print(name)
    if type(name) != str:
        for element in range(0,len(df_panel_all_hitter[name])):
            if math.isinf(df_panel_all_hitter[name].iloc[element]) == True:
                print(str(element) +  '  ' + str(df_panel_all_hitter[name].iloc[element]))

Acronimo
Agente_libre
Altura
Anio
Anio_de_agente_libre
Anios_de_contrato
Antiguedad
At-bats
At-bats_2
Bateos
Bateos_2
Bateos_promedio
Bateos_promedio_2
Bono_por_firma
Cantidad de equipos
Cantidad_agentes_libres
Dobles
Dobles_2
Edad
Edad_al_firmar
Equipo
Estado
Ganancias
Home-runs
Home-runs_2
Juegos
Juegos totales
Juegos_iniciados
Juegos_iniciados_2
Jugador
Pago_efectivo
Pennants won
Peso
Playoffs
Porcentaje_On-base-plus-slugging
Porcentaje_On-base-plus-slugging_2
Porcentaje_juegos
Porcentaje_juegos_iniciados
Porcentaje_juegos_iniciados_2
Porcentaje_on-base
Porcentaje_on-base_2
Porcentaje_slugging
Porcentaje_slugging_2
Posicion
Promedio_victorias
Runs-batted-in
Runs-batted-in_2
Sueldo_ajustado
Sueldo_base
Sueldo_porcentual
Sueldo_regular
TVS
TVS_2
Triples
Triples_2
Valor_contrato_promedio
Valor_contrato_total
Valor_del_contrato
Victorias
WAR
WAR_2
WS ganadas
ln_Sueldo_ajustado
ln_Sueldo_base
ln_Sueldo_regular


In [206]:
for name in df_panel_all_pitcher.columns:
    print(name)
    if type(name) != str:
        for element in range(0,len(df_panel_all_pitcher[name])):
            if math.isinf(df_panel_all_pitcher[name].iloc[element]) == True:
                print(str(element) +  '  ' + str(df_panel_all_pitcher[name].iloc[element]))

Acronimo
Agente_libre
Altura
Anio
Anio_de_agente_libre
Anios_de_contrato
Antiguedad
Bateos_en_contra
Bateos_en_contra_2
Bono_por_firma
Cantidad de equipos
Cantidad_agentes_libres
Carreras_en_contra
Carreras_en_contra_2
Carreras_ganadas
Carreras_ganadas_2
Comando
Comando_2
Control
Control_2
Dominio
Dominio_2
ERA
ERA_2
Edad
Edad_al_firmar
Equipo
Estado
Ganancias
Inning_pitched
Inning_pitched_2
Juegos
Juegos totales
Juegos_iniciados
Jugador
Losses
Losses_2
Pago_efectivo
Pennants won
Peso
Playoffs
Porcentaje_juegos
Posicion
Promedio_victorias
Saves
Saves_2
Strike-outs
Strike-outs_2
Sueldo_ajustado
Sueldo_base
Sueldo_porcentual
Sueldo_regular
TVS
TVS_2
Valor_contrato_promedio
Valor_contrato_total
Valor_del_contrato
Victorias
WAR
WAR_2
WHIP
WHIP_2
WS ganadas
Walks
Walks_2
Wins
Wins_2
ln_Sueldo_ajustado
ln_Sueldo_base
ln_Sueldo_regular


Exportemos los paneles

In [207]:
df_panel_all_hitter.to_csv('ETL_Data/Agent/First_Year_Contract/Period_t/Hitters/All_Hitters/panel_hitters' + '.csv', index = False)
df_panel_all_pitcher.to_csv('ETL_Data/Agent/First_Year_Contract/Period_t/Pitchers/All_Pitchers/panel_pitchers' + '.csv', index = False)

Repetiremos el procedimiento, pero únicamente para quienes son agentes libres

In [208]:
# Inicialización del panel
df_panel_fa_hitter = df_hitters_free_agents[0]
df_panel_fa_pitcher = df_pitchers_free_agents[0]

for year in range(1,period):
    # Hitter
    df_panel_fa_hitter = pd.concat([df_panel_fa_hitter, df_hitters_free_agents[year]])
    
    # Pitcher
    df_panel_fa_pitcher = pd.concat([df_panel_fa_pitcher, df_pitchers_free_agents[year]])

In [209]:
df_panel_fa_hitter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 642 entries, 0 to 34
Data columns (total 68 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Acronimo                            642 non-null    object 
 1   Altura                              642 non-null    float64
 2   Anio                                642 non-null    object 
 3   Anio_de_agente_libre                642 non-null    int64  
 4   Anios_de_contrato                   642 non-null    int64  
 5   Antiguedad                          642 non-null    int64  
 6   At-bats                             642 non-null    int64  
 7   At-bats_2                           642 non-null    int64  
 8   Bateos                              642 non-null    int64  
 9   Bateos_2                            642 non-null    int64  
 10  Bateos_promedio                     642 non-null    float64
 11  Bateos_promedio_2                   642 non-nu

In [210]:
df_panel_fa_pitcher.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 582 entries, 0 to 73
Data columns (total 73 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Acronimo                 582 non-null    object 
 1   Altura                   582 non-null    float64
 2   Anio                     582 non-null    object 
 3   Anio_de_agente_libre     582 non-null    int64  
 4   Anios_de_contrato        582 non-null    int64  
 5   Antiguedad               582 non-null    int64  
 6   Bateos_en_contra         582 non-null    int64  
 7   Bateos_en_contra_2       582 non-null    int64  
 8   Bono_por_firma           582 non-null    int64  
 9   Cantidad de equipos      582 non-null    int64  
 10  Cantidad_agentes_libres  582 non-null    int64  
 11  Carreras_en_contra       582 non-null    int64  
 12  Carreras_en_contra_2     582 non-null    int64  
 13  Carreras_ganadas         582 non-null    int64  
 14  Carreras_ganadas_2       58

In [211]:
df_panel_fa_hitter.to_csv('ETL_Data/Agent/First_Year_Contract/Period_t/Free_Agent/Hitters/panel_hitters' + '.csv', index = False)
df_panel_fa_pitcher.to_csv('ETL_Data/Agent/First_Year_Contract/Period_t/Free_Agent/Pitchers/panel_pitchers' + '.csv', index = False)